In [1]:
!pip install yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 KB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.8/116.8 KB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 9.5 MB/s eta 0:00:00a 0:00:01m
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 KB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 KB 13.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 11.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 KB 26.8 MB/s eta 0:00:00
  Created wheel for peewee: filename=peewee-3.17.5-cp39-cp39-linux_x86_64.whl size=298921 sha256=d6109f12d1ae29716122406cdcf1cf647041bab4998649134777637e63947e76
  Stored in directory: /home/jovyan/

In [90]:
prices_data.index[0]

Timestamp('2004-11-18 00:00:00')

In [111]:
import yfinance as yf
import pandas as pd
import numpy as np

# Fetch historical data once for each asset class
def fetch_all_data(tickers, start, end):
    data = yf.download(tickers, start=start, end=end)['Adj Close']
    return data

# Strategy 1: 100% SPY
def strategy_100_spy(prices, *args, **kwargs):
    portfolio_value = 100
    shares = portfolio_value / prices[0]
    portfolio_values = prices * shares
    return portfolio_values

# Strategy: SPY and alternative asset (cash, gold, bonds) with rebalancing
def strategy_spy_alt(prices, alt_prices, spy_allocation, alt_allocation, rebalance_threshold):
    portfolio_value = 100
    spy_shares = (portfolio_value * spy_allocation) / prices[0]
    alt_shares = (portfolio_value * alt_allocation) / alt_prices[0]
    portfolio_values = []

    for price, alt_price in zip(prices, alt_prices):
        portfolio_value = (spy_shares * price) + (alt_shares * alt_price)
        current_spy_allocation = (spy_shares * price) / portfolio_value
        alt_allocation_current = (alt_shares * alt_price) / portfolio_value

        if abs(current_spy_allocation - spy_allocation) > rebalance_threshold:
            spy_shares = (portfolio_value * spy_allocation) / price
            alt_shares = (portfolio_value * alt_allocation) / alt_price

        portfolio_values.append(portfolio_value)
    
    return portfolio_values

# Run backtests without multiprocessing
def backtest_strategy(strategy, start_dates, period_years, prices_data, alt_ticker=None, **strategy_args):
    results = {}
    for start_date in start_dates:
        end_date = start_date + pd.DateOffset(years=period_years)
        prices = prices_data['SPY'].loc[start_date:end_date]
        if alt_ticker:
            alt_prices = prices_data[alt_ticker].loc[start_date:end_date]
        else:
            alt_prices = prices * 0 + 1  # Dummy prices for 100% SPY strategy
        portfolio_values = strategy(prices, alt_prices, **strategy_args)
        results[start_date.strftime("%Y-%m-%d")] = portfolio_values[-1]
    return results

# Parameters
start_years = range(2000, 2014)
start_dates = [pd.Timestamp(year, month, 1) for year in start_years for month in range(1, 13)]
period_years = 10
cash_allocations = [0.01, 0.05, 0.10, 0.20]
rebalance_thresholds = [0.01, 0.02, 0.03, 0.05]
bond_tickers = ['^TNX', 'VUSTX', 'VFISX']  # 10-Year Treasury Yield, Long-Term Treasury Fund, Short-Term Treasury Fund

# Fetch data
all_tickers = ['SPY', 'GC=F'] + bond_tickers
start_date = '1999-01-01'
end_date = '2024-01-01'
prices_data = fetch_all_data(all_tickers, start=start_date, end=end_date)

# Backtest the strategies
results = {}
results['100% SPY'] = backtest_strategy(strategy_100_spy, start_dates, period_years, prices_data)

# SPY and Cash
for cash_allocation in cash_allocations:
    for rebalance_threshold in rebalance_thresholds:
        if rebalance_threshold <= cash_allocation:
            spy_allocation = 1 - cash_allocation
            strategy_name = f"{int(cash_allocation * 100)}% Cash, Rebalance {int(rebalance_threshold * 100)}%"
            results[strategy_name] = backtest_strategy(strategy_spy_alt, start_dates, period_years, prices_data, None, spy_allocation=spy_allocation, alt_allocation=cash_allocation, rebalance_threshold=rebalance_threshold)

# SPY and Gold
for gold_allocation in cash_allocations:
    for rebalance_threshold in rebalance_thresholds:
        if rebalance_threshold <= gold_allocation:
            spy_allocation = 1 - gold_allocation
            strategy_name = f"{int(gold_allocation * 100)}% Gold, Rebalance {int(rebalance_threshold * 100)}%"
            results[strategy_name] = backtest_strategy(strategy_spy_alt, start_dates, period_years, prices_data, 'GC=F', spy_allocation=spy_allocation, alt_allocation=gold_allocation, rebalance_threshold=rebalance_threshold)

# SPY and Bonds
for bond_ticker in bond_tickers:
    for bond_allocation in cash_allocations:
        for rebalance_threshold in rebalance_thresholds:
            if rebalance_threshold <= bond_allocation:
                spy_allocation = 1 - bond_allocation
                strategy_name = f"{int(bond_allocation * 100)}% {bond_ticker}, Rebalance {int(rebalance_threshold * 100)}%"
                results[strategy_name] = backtest_strategy(strategy_spy_alt, start_dates, period_years, prices_data, bond_ticker, spy_allocation=spy_allocation, alt_allocation=bond_allocation, rebalance_threshold=rebalance_threshold)

# Convert results to DataFrame
df_results = pd.DataFrame(results).transpose()






[*********************100%%**********************]  5 of 5 completed


Average return for start dates 2000-2010:
Series([], dtype: float64)

Average return for start dates 2011-2013:
Series([], dtype: float64)


In [118]:
# Calculate average returns
def calculate_average_return(df, start_years):
    valid_columns = [col for col in df.columns if pd.Timestamp(col).year in start_years]
    filtered_df = df[valid_columns].dropna(axis=1, how='any')
    return filtered_df.mean(axis=1)

average_return_2000_2010 = calculate_average_return(df_results, [it.year for it in pd.date_range(start="2000-01-01", end="2010-12-31")])
average_return_2011_2013 = calculate_average_return(df_results, [it.year for it in pd.date_range(start="2011-01-01", end="2013-12-31")])

print("Improvement over 100% SPY  for start dates 2000-2010:")
print(average_return_2000_2010.sort_values(ascending=False) / average_return_2000_2010["100% SPY"])
print("\n Improvement over 100% SPY for start dates 2011-2013:")
print(average_return_2011_2013.sort_values(ascending=False) / average_return_2011_2013["100% SPY"])

Improvement over 100% SPY  for start dates 2000-2010:
20% Gold, Rebalance 1%     1.059955
20% Gold, Rebalance 3%     1.053824
20% Gold, Rebalance 5%     1.051098
20% Gold, Rebalance 2%     1.049904
10% Gold, Rebalance 5%     1.037210
10% Gold, Rebalance 3%     1.024495
10% Gold, Rebalance 2%     1.023749
10% Gold, Rebalance 1%     1.023549
5% Gold, Rebalance 5%      1.022085
5% Gold, Rebalance 3%      1.020343
5% Gold, Rebalance 2%      1.015465
20% VUSTX, Rebalance 1%    1.014121
                             ...   
10% ^TNX, Rebalance 2%     0.904227
10% ^TNX, Rebalance 5%     0.903456
10% ^TNX, Rebalance 3%     0.903229
10% ^TNX, Rebalance 1%     0.899149
20% Cash, Rebalance 5%     0.874039
20% Cash, Rebalance 3%     0.868208
20% Cash, Rebalance 2%     0.865998
20% Cash, Rebalance 1%     0.864311
20% ^TNX, Rebalance 5%     0.806638
20% ^TNX, Rebalance 3%     0.804300
20% ^TNX, Rebalance 1%     0.800529
20% ^TNX, Rebalance 2%     0.800042
Length: 66, dtype: float64

 Improvement over 

In [116]:
average_return_2000_2010["100% SPY"]

228.09623594483458

In [112]:
df_results.mean(axis=1).sort_values(ascending=False)

5% Gold, Rebalance 5%      259.647821
5% Gold, Rebalance 3%      258.932669
1% Gold, Rebalance 1%      258.760327
10% Gold, Rebalance 5%     258.495038
5% Gold, Rebalance 2%      257.965637
5% Gold, Rebalance 1%      257.175671
10% Gold, Rebalance 3%     256.087496
10% Gold, Rebalance 2%     255.635252
10% Gold, Rebalance 1%     255.610839
20% Gold, Rebalance 1%     255.029576
20% Gold, Rebalance 5%     254.112923
20% Gold, Rebalance 3%     254.042123
                              ...    
20% VFISX, Rebalance 5%    222.161511
20% VFISX, Rebalance 3%    220.372477
20% VFISX, Rebalance 1%    219.602476
20% VFISX, Rebalance 2%    219.556746
20% ^TNX, Rebalance 5%     214.494093
20% Cash, Rebalance 5%     213.973885
20% ^TNX, Rebalance 3%     213.441796
20% Cash, Rebalance 3%     212.264362
20% Cash, Rebalance 2%     211.529609
20% ^TNX, Rebalance 2%     211.508153
20% ^TNX, Rebalance 1%     211.505543
20% Cash, Rebalance 1%     211.076613
Length: 66, dtype: float64

In [87]:
prices_data.dropna()

Ticker,AGG,GLD,SHY,SPY,TLT
Date,,,,,
2004-11-18,56.108089,44.380001,58.552788,81.896454,48.101654
2004-11-19,55.993465,44.779999,58.459660,80.986061,47.717758
2004-11-22,56.080830,44.950001,58.474010,81.372269,47.966442
2004-11-23,56.048061,44.750000,58.466881,81.496452,48.025940
2004-11-24,56.102665,45.049999,58.459660,81.689537,48.025940
2004-11-26,55.938824,45.290001,58.402355,81.627457,47.712383
2004-11-29,55.785946,45.400002,58.409534,81.255051,47.204178
2004-11-30,55.813274,45.119999,58.423862,81.310204,47.004135
2004-12-01,55.603924,45.380001,58.463360,82.234428,46.930805


In [69]:
results

{'100% SPY': {},
 '1% Cash, Rebalance 1%': {},
 '5% Cash, Rebalance 1%': {},
 '5% Cash, Rebalance 2%': {},
 '5% Cash, Rebalance 3%': {},
 '5% Cash, Rebalance 5%': {},
 '10% Cash, Rebalance 1%': {},
 '10% Cash, Rebalance 2%': {},
 '10% Cash, Rebalance 3%': {},
 '10% Cash, Rebalance 5%': {},
 '20% Cash, Rebalance 1%': {},
 '20% Cash, Rebalance 2%': {},
 '20% Cash, Rebalance 3%': {},
 '20% Cash, Rebalance 5%': {},
 '1% Gold, Rebalance 1%': {},
 '5% Gold, Rebalance 1%': {},
 '5% Gold, Rebalance 2%': {},
 '5% Gold, Rebalance 3%': {},
 '5% Gold, Rebalance 5%': {},
 '10% Gold, Rebalance 1%': {},
 '10% Gold, Rebalance 2%': {},
 '10% Gold, Rebalance 3%': {},
 '10% Gold, Rebalance 5%': {},
 '20% Gold, Rebalance 1%': {},
 '20% Gold, Rebalance 2%': {},
 '20% Gold, Rebalance 3%': {},
 '20% Gold, Rebalance 5%': {},
 '1% AGG, Rebalance 1%': {},
 '5% AGG, Rebalance 1%': {},
 '5% AGG, Rebalance 2%': {},
 '5% AGG, Rebalance 3%': {},
 '5% AGG, Rebalance 5%': {},
 '10% AGG, Rebalance 1%': {},
 '10% AGG, R

In [48]:
df_results.mean(axis=1).sort_values(ascending=False).head(30)

1% Gold, Rebalance 1%     297.231736
5% Gold, Rebalance 5%     293.698314
5% Gold, Rebalance 3%     293.017015
5% Gold, Rebalance 2%     291.687757
5% Gold, Rebalance 1%     290.652836
1% AGG, Rebalance 1%      286.534301
10% Gold, Rebalance 5%    286.305337
10% Gold, Rebalance 3%    283.631327
10% Gold, Rebalance 2%    282.687755
10% Gold, Rebalance 1%    282.631017
5% AGG, Rebalance 3%      281.878849
5% AGG, Rebalance 5%      281.321905
5% AGG, Rebalance 2%      279.953234
5% AGG, Rebalance 1%      279.390210
1% TLT, Rebalance 1%      278.539489
1% SHY, Rebalance 1%      277.669412
5% TLT, Rebalance 5%      276.880195
5% TLT, Rebalance 2%      276.257543
5% TLT, Rebalance 1%      276.009914
5% TLT, Rebalance 3%      275.612223
10% AGG, Rebalance 5%     274.085300
10% TLT, Rebalance 5%     273.221939
10% TLT, Rebalance 1%     272.326652
10% TLT, Rebalance 3%     272.212687
10% TLT, Rebalance 2%     272.150361
5% SHY, Rebalance 5%      272.025082
10% AGG, Rebalance 3%     271.887443
5